<a href="https://colab.research.google.com/github/fjgr/IA_BigData/blob/main/M2D/Tarea-2/M2D_Tarea_2_Algoritmos_Gen%C3%A9ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# Importar librerías necesarias
import random
import pandas as pd

# Datos de las películas
peliculas = [
    {"nombre": "La última casa a la izquierda", "genero": "TERROR", "tamaño": 1.830},
    {"nombre": "Saw IV", "genero": "TERROR", "tamaño": 1.435},
    {"nombre": "La huérfana", "genero": "TERROR", "tamaño": 2.163},
    {"nombre": "Furia de Titanes", "genero": "ACCIÓN", "tamaño": 1.746},
    {"nombre": "El hombre de acero", "genero": "ACCIÓN", "tamaño": 0.964},
    {"nombre": "Los Vengadores", "genero": "ACCIÓN", "tamaño": 2.032},
    {"nombre": "American Pie: El reencuentro", "genero": "COMEDIA", "tamaño": 1.257},
    {"nombre": "El lado bueno de las cosas", "genero": "COMEDIA", "tamaño": 3.139},
    {"nombre": "Los tres chiflados", "genero": "COMEDIA", "tamaño": 0.750},
    {"nombre": "Jugada salvaje", "genero": "SUSPENSE", "tamaño": 2.275},
    {"nombre": "El cuerpo", "genero": "SUSPENSE", "tamaño": 2.082},
    {"nombre": "15 años y un día", "genero": "SUSPENSE", "tamaño": 2.321},
]

# Restricciones
restricciones = {
    "género": {"COMEDIA": "TERROR"},
    "pares_prohibidos": [
        {"nombre1": "Jugada salvaje", "nombre2": "El cuerpo"},
        {"nombre1": "Furia de Titanes", "nombre2": "El hombre de acero"},
    ],
}

# Tamaños de discos
tamaños_dvd = [4.7, 8.5, 13.3, 15.9]

# Hiperparámetros del algoritmo genético
POBLACIÓN_INICIAL = 100
GENERACIONES = 100
PROB_MUTACIÓN = 0.1
PROB_RECOMBINACIÓN = 0.8

# Función de fitness
def calcular_fitness(individuo, tamaño_objetivo):
    tamaño_total = sum(p["tamaño"] for i, p in enumerate(peliculas) if individuo[i] == 1)
    penalización = 0

    # Restricción 1: No mezclar géneros prohibidos
    géneros = set(p["genero"] for i, p in enumerate(peliculas) if individuo[i] == 1)
    for g1, g2 in restricciones["género"].items():
        if g1 in géneros and g2 in géneros:
            penalización += 1000

    # Restricción 2: Pares prohibidos
    seleccionadas = [p["nombre"] for i, p in enumerate(peliculas) if individuo[i] == 1]
    for restr in restricciones["pares_prohibidos"]:
        if restr["nombre1"] in seleccionadas and restr["nombre2"] in seleccionadas:
            penalización += 1000

    # Evaluación del espacio utilizado
    diferencia = abs(tamaño_objetivo - tamaño_total)
    if tamaño_total > tamaño_objetivo:
        penalización += 100 * (tamaño_total - tamaño_objetivo)

    return -diferencia - penalización

# Crear una población inicial
def crear_población():
    return [
        [random.randint(0, 1) for _ in range(len(peliculas))]
        for _ in range(POBLACIÓN_INICIAL)
    ]

# Selección por torneo
def seleccion_torneo(población, fitness, tamaño_torneo=3):
    mejor = None
    for _ in range(tamaño_torneo):
        individuo = random.choice(población)
        individuo_tuple = tuple(individuo)  # Convertir lista a tupla
        if mejor is None or fitness[individuo_tuple] > fitness[mejor]:
            mejor = individuo_tuple
    return list(mejor)  # Convertir de vuelta a lista

# Cruce de un punto
def recombinación(parent1, parent2):
    punto = random.randint(1, len(parent1) - 1)
    return parent1[:punto] + parent2[punto:]

# Mutación simple
def mutación(individuo):
    i = random.randint(0, len(individuo) - 1)
    individuo[i] = 1 - individuo[i]
    return individuo

# Algoritmo genético principal
def algoritmo_genético(tamaño_dvd):
    # Crear población inicial
    población = crear_población()

    for _ in range(GENERACIONES):
        fitness = {
            tuple(ind): calcular_fitness(ind, tamaño_dvd) for ind in población
        }

        # Nueva generación
        nueva_población = []

        while len(nueva_población) < len(población):
            # Selección
            padre1 = seleccion_torneo(población, fitness)
            padre2 = seleccion_torneo(población, fitness)

            # Recombinación
            if random.random() < PROB_RECOMBINACIÓN:
                hijo = recombinación(padre1, padre2)
            else:
                hijo = padre1[:]

            # Mutación
            if random.random() < PROB_MUTACIÓN:
                hijo = mutación(hijo)

            nueva_población.append(hijo)

        población = nueva_población

    # Mejor individuo final
    mejor_individuo = max(población, key=lambda ind: calcular_fitness(ind, tamaño_dvd))
    tamaño_total = sum(
        p["tamaño"] for i, p in enumerate(peliculas) if mejor_individuo[i] == 1
    )
    seleccionadas = [p["nombre"] for i, p in enumerate(peliculas) if mejor_individuo[i] == 1]

    return seleccionadas, tamaño_total

# Ejecutar el algoritmo para cada tamaño de DVD
resultados = {tam: algoritmo_genético(tam) for tam in tamaños_dvd}

# Mostrar resultados en DataFrame
df_resultados = pd.DataFrame(
    [
        {
            "Tamaño DVD": tam,
            "Películas Seleccionadas": resultados[tam][0],
            "Tamaño Total": resultados[tam][1],
        }
        for tam in resultados
    ]
)

# Mostrar resultados
print(df_resultados)


   Tamaño DVD                            Películas Seleccionadas  Tamaño Total
0         4.7       [Saw IV, El hombre de acero, Jugada salvaje]         4.674
1         8.5  [El hombre de acero, American Pie: El reencuen...         8.431
2        13.3  [La última casa a la izquierda, Saw IV, La hué...        12.827
3        15.9  [La última casa a la izquierda, Saw IV, La hué...        13.802
